In [1]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [13]:
import os

if not os.path.isdir("data"):
  print("No data folder. Building all")
  !pip uninstall tensorflow -y
  !pip install tensorflow==2.0.0
  !7za x /content/gdrive/My\ Drive/Puc/Projeto\ Final/Datasets/mention_training.7z -odata

No data folder. Building all
Uninstalling tensorflow-2.0.0:
  Successfully uninstalled tensorflow-2.0.0
  Using cached https://files.pythonhosted.org/packages/46/0f/7bd55361168bb32796b360ad15a25de6966c9c1beb58a8e30c01c8279862/tensorflow-2.0.0-cp36-cp36m-manylinux2010_x86_64.whl



7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/My Drive/Puc/Projeto Final/Datasets/                                                              1 file, 445369272 bytes (425 MiB)

Extracting archive: /content/gdrive/My Drive/Puc/Projeto Final/Datasets/mention_training.7z
--
Path = /content/gdrive/My Drive/Puc/Projeto Final/Datasets/mention_training.7z
Type = 7z
Physical Size = 445369272
Headers Size = 28843
Method = LZMA2:24
Solid = +
Blocks = 4

  0%      0% 13 - mention_training/bc_cctv_00_cctv_0002_003.tsv                                                       

In [0]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.backend as K

In [0]:
MAX_NUM_MENTIONS = 20
MAX_NUM_PREDICTIONS = int(MAX_NUM_MENTIONS * (MAX_NUM_MENTIONS - 1) / 2)
NUM_CPU_READ = 4
INPUT_EMBEDDINGS_SIZE = 768
BUFFER_SIZE = 100

EPOCHS = 5
VALIDATION_STEPS = 10
BATCH_SIZE = 20

DATA_FOLDER="data/mention_training"

In [0]:
def input_fn_builder(input_files,
                     max_seq_length,
                     max_predictions_per_seq,
                     is_training,
                     num_cpu_threads=NUM_CPU_READ):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        name_to_features = {
            "input_embeddings":
                tf.io.FixedLenFeature(max_seq_length * INPUT_EMBEDDINGS_SIZE, tf.float32),
            "input_mask":
                tf.io.FixedLenFeature(max_seq_length, tf.int64),
            "labels":
                tf.io.FixedLenFeature(max_predictions_per_seq, tf.int64),
            "output_mask":
                tf.io.FixedLenFeature(max_predictions_per_seq, tf.int64)
        }

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        if is_training:
            d = tf.data.Dataset.from_tensor_slices(tf.constant(input_files))
            d = d.repeat()
            d = d.shuffle(buffer_size=len(input_files))

            # `cycle_length` is the number of parallel files that get read.
            cycle_length = min(num_cpu_threads, len(input_files))

            # `sloppy` mode means that the interleaving is not exact. This adds
            # even more randomness to the training pipeline.
            d = d.apply(
                tf.data.experimental.parallel_interleave(
                    tf.data.TFRecordDataset,
                    sloppy=is_training,
                    cycle_length=cycle_length))
            d = d.shuffle(buffer_size=BUFFER_SIZE)
        else:
            d = tf.data.TFRecordDataset(input_files)
            # Since we evaluate for a fixed number of steps we don't want to encounter
            # out-of-range exceptions.
            d = d.repeat()

        # We must `drop_remainder` on training because the TPU requires fixed
        # size dimensions. For eval, we assume we are evaluating on the CPU or GPU
        # and we *don't* want to drop the remainder, otherwise we wont cover
        # every sample.
        d = d.apply(
            tf.data.experimental.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                num_parallel_batches=num_cpu_threads,
                drop_remainder=True))
        return d

    return input_fn

@tf.function
def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.io.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.cast(t, tf.int32)
        example[name] = t

    embeddings = tf.reshape(example["input_embeddings"], (MAX_NUM_MENTIONS, INPUT_EMBEDDINGS_SIZE))
    mask = tf.cast(tf.reshape(example["output_mask"], (190, 1)), tf.float32)
    Y = tf.one_hot(example["labels"], depth=2)
    Y = tf.multiply(Y, mask)

    print(f"Y Shape:{Y.shape}")

    return {"input_embeddings": embeddings,
            "output_mask": mask}, Y


def slice_and_split_input(data_folder):
    if isinstance(data_folder, str):
        # One folder -> dev, train/test split
        files = os.listdir(data_folder)
        if len(files) == 0:
            print(f"No files found in folder {data_folder}")
            return
        files = [f"{data_folder}/{f}" for f in files if f.endswith(".tsv")]
        train_files, valid_files = train_test_split(files)

        print(f"Using {len(train_files)} files for train and {len(valid_files)} for validation")

    else:
        # Two folders -> train/test already splitted
        train_files = os.listdir(data_folder[0])
        train_files = [f"{data_folder}/{f}" for f in train_files if f.endswith(".tsv")]

        valid_files = os.listdir(data_folder[1])
        valid_files = [f"{data_folder}/{f}" for f in valid_files if f.endswith(".tsv")]

    train_input = input_fn_builder(train_files,
                                   MAX_NUM_MENTIONS,
                                   MAX_NUM_PREDICTIONS,
                                   True)
    valid_input = input_fn_builder(valid_files,
                                   MAX_NUM_MENTIONS,
                                   MAX_NUM_PREDICTIONS,
                                   False)
    return train_input, valid_input

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def get_model():
  def create_model():
    embeddings = tf.keras.layers.Input(shape=(20, 768,), name="input_embeddings")
    output_mask = tf.keras.layers.Input(shape=(190, 1), name="output_mask")

    h = tf.keras.layers.Flatten()(embeddings)
    #h = tf.keras.layers.Dense(20 * 768, activation='relu')(h)
    #h = tf.keras.layers.Dropout(0.2)(h)
    h = tf.keras.layers.Dense(190 * 100, activation='relu')(h)    
    h = tf.keras.layers.Reshape((190, 100))(h)
    e = h
    h = tf.keras.layers.Conv1D(2, 1, strides=1)(h)
    h = tf.keras.layers.Activation('softmax')(h)

    h = tf.keras.layers.Multiply()([h, output_mask])

    o = h
    #task_loss = Lambda(lambda x: x[0] * masked_classification_loss(x[1], x[2], x[3]), name=task.name + '_loss')([task_target, logits, task_mask])

    model = tf.keras.Model([embeddings, output_mask], o)
    model.summary()

    encoder = tf.keras.Model(embeddings, e)
    encoder.summary()

    return model, encoder

  def f1_score(y_true, y_pred):
      # Count positive samples.
      c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      c2 = K.sum(K.round(K.clip(y_pred, 0, 1))) + 1e-5
      c3 = K.sum(K.round(K.clip(y_true, 0, 1))) + 1e-5

      # If there are no true samples, fix the F1 score at 0.
      #if c3 == 0:
      #    return 0.0

      # How many selected items are relevant?
      precision = c1 / c2

      # How many relevant items are selected?
      recall = c1 / c3

      # Calculate f1_score
      f1_score = 2 * (precision * recall) / (precision + recall + 1e-5)
      return f1_score


  def my_loss(y_true, y_pred):
    denom = tf.keras.backend.sum(output_mask) + 1e-5
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

  if True: # skip TPU
    model,encoder = create_model()
    model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy', f1_score])
    return model, encoder
  else:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_host(resolver.master())
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

    with strategy.scope():
        model = create_model()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss=tf.keras.losses.binary_crossentropy,
                      metrics=[tf.keras.metrics.categorical_accuracy, f1_score])
        return model

def get_data(folder):
    train_input_fn, eval_input_fn = slice_and_split_input(folder)

    train_data = train_input_fn({"batch_size": BATCH_SIZE})
    eval_data = eval_input_fn({"batch_size": BATCH_SIZE})
    return train_data, eval_data

In [10]:
model, encoder = get_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_embeddings (InputLayer)   [(None, 20, 768)]    0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 15360)        0           input_embeddings[0][0]           
__________________________________________________________________________________________________
dense (Dense)                   (None, 19000)        291859000   flatten[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 190, 100)     0           dense[0][0]                      
______________________________________________________________________________________________

In [0]:
encoder.load_weights('/content/gdrive/My Drive/Puc/Projeto Final/encoder.h5')

In [17]:
train_data, eval_data = get_data(DATA_FOLDER)

Using 2081 files for train and 694 for validation
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
Y Shape:(190, 2)


In [0]:
enc = encoder.predict(eval_data, steps=10)

In [19]:
enc[5][0]

array([ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       , 21.701864 ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  3.8208017,  0.       ,  0.       ,
       16.91652  ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        2.1621614,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       , 10.546637 ,  0.    